<h1>NumPy array File Streaming</h1><br>
numpy를 이용하여 임의의 정수 행렬(4x4)를 생성하고 화면에 표시한다.<br>
그 행렬을 파일에 저장한 후에, 다시 로드하여 다시 화면에 표시한다.

In [29]:
import numpy as np
arr1 = np.random.randint(1,21,16).reshape(4,4)
print('arr1:\n',arr1)

#np.save('arr1',arr1)
arr2 = np.load('arr1.npy')
print('\narr2 -파일에서 로드한 arr1-\n',arr2)

arr1:
 [[ 6  4 17 19]
 [18  7  4 19]
 [13  5 12  7]
 [ 8 17 19  8]]

arr2 -파일에서 로드한 arr1-
 [[18 20  2  4]
 [ 5 10  2  4]
 [13 13 15  4]
 [ 3 10  9 18]]


In [49]:
# Structured array로 테스트
struct = np.array([(11,'Ward','010-2532-1245'),
                  (12,'Olie','010-4535-6875'),
                  (13,'David','010-5985-5831')],
                 dtype=[('num','i2'),('name','U6'),('phone','a15')])
np.save('user',struct)
struct2 = np.load('user.npy')
print(struct2)

#isnan 사용예
arr = np.arange(0,5)
#arr[:]= np.nan  # 정수공간에 np.nan을 설정할 수 없음
print(arr.dtype) # int32
print(np.isnan(arr)) # [False False False False False]

s_arr = arr.astype(np.str) # 문자열 배열로 자료형 변환
s_arr[0] = np.nan # 문자열 공간에는 np.nan 설정 가능
#s_arr[:] = np.nan
print(s_arr)

s_arr[s_arr=='nan'] = 0  # 'nan'을 '0'으로 변환
s_arr.astype(np.int)    # 자료형은 int형으로 변환

[(11, 'Ward', b'010-2532-1245') (12, 'Olie', b'010-4535-6875')
 (13, 'David', b'010-5985-5831')]
int32
[False False False False False]
['nan' '1' '2' '3' '4']


array([0, 1, 2, 3, 4])

<h2>np.genfromtxt()로 csv파일 읽기</h2>

In [48]:
import pandas as pd

data2 = np.genfromtxt('ch3_data.csv',delimiter=',',names=True)
print(data2)
#df = pd.read_csv('ch3_data.csv')    # pandas 모듈 사용하는 방법
#df.fillna(0, inplace=True)
#df
v1 = data2['value1']
v1[np.isnan(v1)] = 0    # 불리언 인덱스 추출할 용도로 np.isnan() 활용

v2 = data2['value2']
v2[np.isnan(v2)] = 0

print(data2)

[(1., 123.,  1.4, 23.) (2.,  nan, 21.4, 18.) (3., 164.,  nan, 19.)]
[(1., 123.,  1.4, 23.) (2.,   0., 21.4, 18.) (3., 164.,  0. , 19.)]


<h1>NumPy 종합 실습</h1><br>
employee.csv에 5명분의 데이터 저장<br>
필드명: num, name, dep, sal, hiredate<br><br>

메뉴: 목록(s), 검색(f), 추가(a), 수정(u), 삭제(d), 종료(x)<br>
<br>
번호로 검색<br>
부서번호로 검색<br>
- 부서별 급여평균/합계(부서별 검색시 제시)

In [50]:
import numpy as np

while True:
    tot = np.genfromtxt('employee.csv',delimiter=',',
                    dtype=[('num','i2'),('name','U12'),('dep','U2'),('sal','i2'),('hiredate','U10')])
    comm = input('\n----명령어를 입력하세요(검색-f, 추가-a, 수정-u, 삭제-d, 종료-x): ')
    if comm == 's':
        print(tot)
    elif comm == 'f':
        print('<<<<멤버 검색>>>>')
        wayToFind = input(' 번호로 검색하시려면 1, 부서로 검색하시려면 2를 입력하세요:')
        if wayToFind == '1':
            fnum = input(' 검색할 번호를 입력하세요 :')
            for mem in tot:
                if mem['num']==int(fnum):
                    print(mem)
        elif wayToFind == '2':
            dep = input(' 검색할 부서명을 입력하세요 :')
            print(f'\n-----{dep}부서 명단------')
            for mem in tot:
                if mem['dep'] == dep:
                    print(mem)
            salMean = np.mean(tot['sal'][tot['dep']==dep])
            salTot = np.sum(tot['sal'][tot['dep']==dep])
            print(f'평균급여:{salMean}\n급여합계:{salTot}')
        else:
            print('1 또는 2를 입력하세요')
    elif comm == 'a':
        print('<<<<멤버 추가>>>>')
        info = input(' 번호 이름 부서 연봉 입사일을 띄어서 입력하세요\n')
        if(len(info.split())!=5):
            print('형식에 맞추어 입력하세요')
        else:
            tnum, tname, tdep, tsal, thiredate = int(info.split()[0]), info.split()[1], info.split()[2], int(info.split()[3]), info.split()[4]
            tmem = np.array([(tnum, tname, tdep, tsal, thiredate)],dtype=[('num','i2'),('name','U12'),('dep','U2'),('sal','i2'),('hiredate','U10')])
            tot = np.hstack((tot,tmem))
            np.savetxt('employee.csv',tot,delimiter=',',fmt='%d,%s,%s,%d,%s')
            print('저장 완료!')
    elif comm == 'u':
        print('<<<<멤버 수정>>>>')
        fnum = input(' 수정할 번호를 입력하세요 :')
        for mem in tot:
            if mem['num']==int(fnum):
                info = input(' 변경할 이름과 부서를 띄어서 입력하세요\n')
                mem['name'], mem['dep'] = info.split(' ')
                np.savetxt('employee.csv',tot,delimiter=',',fmt='%d,%s,%s,%d,%s')
                print('수정 완료!')
    elif comm == 'd':
        print('<<<<멤버 삭제>>>>')
        fnum = input('삭제할 번호를 입력하세요 :')
        
        tot = np.delete(tot, np.where(tot['num']==int(fnum)))
        np.savetxt('employee.csv',tot,delimiter=',',fmt='%d,%s,%s,%d,%s')
        print('삭제 완료!')
    elif comm == 'x':
        break
print('\n[[[[[[[[[  프로그램 종료~(^.^)~~~  ]]]]]]]]]')


----명령어를 입력하세요(검색-f, 추가-a, 수정-u, 삭제-d, 종료-x): x

[[[[[[[[[  프로그램 종료~^.^~~~  ]]]]]]]]]
